In [ ]:
import pandas as pd
from EventEnum import UsstEventType
import os

In [ ]:

TaskStart = {}
ToleranteTime = {}


reader = pd.read_csv(r'task6.csv',header= None,chunksize=10000)
index = 0
for Chunk in reader:
    for line in Chunk.values.tolist():
        TaskStart[index] = float(line[0])
        ToleranteTime[index] = float(line[1])
        index += 1


In [ ]:
outPath = '4_4_5_200_1'
# a_b_c_d_e
# a和b表示切分后的卫星分片的规模
# c表示的是使用的任务集序号
# d是表示一秒内产生的任务数量
# e地面站数量
if not os.path.exists(outPath):
    os.makedirs(outPath)

In [ ]:
# reader = pd.read_csv(r'out\main\4_4_4_400.csv',header= None,chunksize=10000)

Path = r'out\20240529_10\main'

# 获取目录下的所有文件
files = os.listdir(Path)

# 过滤出CSV文件
csv_files = [f for f in files if f.endswith('.csv')]

TaskCostTime = {}
GetTargetCostTime = {}
waitProcess = {}
TouchAllcTime = {}
ProcessCostTime = {}
TransmiteCostTime = {}

TaskPreProcess = {}
TaskFinsh = {}

processNode = {}
allocationNode = {}

for csv_path in csv_files:
    reader = pd.read_csv(os.path.join(Path,csv_path),header= None,chunksize=3000)
    for Chunk in reader:
        for line in Chunk.values.tolist():
            TaskID = int(line[0])
            EventType = UsstEventType(int(line[2]))
            if EventType == UsstEventType.Transmission:
                TransmiteCostTime.setdefault(TaskID,0)
                TransmiteCostTime[TaskID] += float(line[4]-line[3])
            elif EventType == UsstEventType.Allocation:
                LocNode = int(line[-1]) 
                startTime = float(line[3])
                endTime = float(line[4])
                allocationNode.setdefault(LocNode,0)
                allocationNode[LocNode] += 1
                TouchAllcTime[TaskID] = startTime - TaskStart[TaskID]
                GetTargetCostTime[TaskID] = endTime - TaskStart[TaskID]
            elif EventType == UsstEventType.waitProcess:
                startTime = float(line[3])
                endTime = float(line[4])
                waitProcess[TaskID] = endTime - startTime
            elif EventType == UsstEventType.preProcess:
                endTime = float(line[4])
                TaskPreProcess[TaskID] = endTime - TaskStart[TaskID]
            elif EventType == UsstEventType.Process:
                LocNode = int(line[-1]) 
                endTime = float(line[4])
                startTime = float(line[3])
                ProcessCostTime[TaskID] = endTime - startTime
                processNode.setdefault(LocNode,0)
                processNode[LocNode] += 1
            elif EventType == UsstEventType.End:
                endTime = float(line[4])
                TaskFinsh[TaskID] = endTime - TaskStart[TaskID]


以下是整个任务完成消耗时间

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

SortedTaskFinsh = sorted(TaskFinsh.items())
x = [TaskStart[a[0]] for a in SortedTaskFinsh]
y = [a[1]*1000 for a in SortedTaskFinsh]



plt.scatter(x,y,s=1)
# plt.show()

average = np.mean(y)
print(average)
plt.savefig(f'{outPath}/{average}TaskFinsh.png')

以下是任务获得控制节点分配结果的延迟

In [ ]:
plt.cla()

Sorted = sorted(GetTargetCostTime.items())
x = [TaskStart[a[0]] for a in Sorted]
y = [a[1]*1000 for a in Sorted]

plt.scatter(x,y,s=0.2)
# plt.show()

average = np.mean(y)
print(average)
plt.savefig(f'{outPath}/{average}GetTargetCostTime.png')

以下是任务到达控制节点的延迟

In [ ]:
plt.cla()

Sorted = sorted(TouchAllcTime.items())
x = [TaskStart[a[0]] for a in Sorted]
y = [a[1]*1000 for a in Sorted]

plt.scatter(x,y,s=0.2)
# plt.show()

average = np.mean(y)
print(average)
plt.savefig(f'{outPath}/{average}TouchAllcTime.png')

In [ ]:
plt.cla()

Sorted = sorted(waitProcess.items())
x = [TaskStart[a[0]] for a in Sorted]
y = [a[1]*1000 for a in Sorted]

plt.scatter(x,y,s=5)
# plt.show()

average = np.mean(y)
print(average)
plt.savefig(f'{outPath}/{average}waitProcess.png')

In [ ]:
plt.cla()

Sorted = sorted(ProcessCostTime.items())
x = [TaskStart[a[0]] for a in Sorted]
y = [a[1]*1000 for a in Sorted]

plt.scatter(x,y,s=0.5)
# plt.show()

average = np.mean(y)
print(average)
plt.savefig(f'{outPath}/{average}ProcessCostTime.png')

In [ ]:
plt.cla()

Sorted = sorted(TransmiteCostTime.items())
x = [TaskStart[a[0]] for a in Sorted]
y = [a[1]*1000 for a in Sorted]

plt.scatter(x,y,s=5)
# plt.show()

average = np.mean(y)
print(average)
plt.savefig(f'{outPath}/{average}TransmiteCostTime.png')

In [ ]:
plt.cla()

maxTime = max(processNode.values())

for i in range(144):
    x = i // 12 + 1
    y = i % 12 + 1
    if i in processNode:
        color = (1 - processNode[i]/maxTime,0.8,0.8)
    else:
        color = (1,0.8,0.8)
    plt.scatter(x, y, color=color)

plt.xlim(0, 13)  # 36
plt.ylim(0, 13)  # 20
plt.savefig(f'{outPath}/processNode.png')

In [ ]:
plt.cla()

maxTime = max(allocationNode.values())

for i in range(144):
    x = i // 12 + 1
    y = i % 12 + 1
    if i in allocationNode:
        color = (1 - allocationNode[i]/maxTime,0.8,0.8)
    else:
        color = (1,0.8,0.8)
    plt.scatter(x, y, color=color)

plt.xlim(0, 13)  # 36
plt.ylim(0, 13)  # 20
plt.savefig(f'{outPath}/allocationNode.png')